In [1]:
import paho.mqtt.client as mqtt
import os
import pywt
import librosa
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib

# Constants for feature extraction
N_MFCC = 13
N_FFT = 2048
HOP_LENGTH = 512

# Load the trained SVM model
model_filename = 'suara_model_svm.pkl'
svm_model = joblib.load(model_filename)

# Function for denoising using wavelet
def denoise_wavelet(audio, threshold=1e-6, preserve_threshold=0.05):
    coeffs = pywt.wavedec(audio, 'db1', level=6)
    preserved_coeffs = [coeffs[0]]
    
    for i in range(1, len(coeffs)):
        thresholded_coeff = pywt.threshold(coeffs[i], threshold, mode='soft')
        if np.sum(np.abs(thresholded_coeff)) > preserve_threshold:
            preserved_coeffs.append(thresholded_coeff)
        else:
            preserved_coeffs.append(np.zeros_like(thresholded_coeff))

    audio_denoised = pywt.waverec(preserved_coeffs, 'db1')
    return audio_denoised

# Function to preprocess audio and extract features
def preprocess_audio(audio_data, n_mfcc=N_MFCC, n_fft=N_FFT, hop_length=HOP_LENGTH):

    # Denoising using wavelet
    audio = librosa.util.normalize(audio_data)
    audio_denoised = denoise_wavelet(audio, preserve_threshold=0.05)

    # Extracting features
    mfccs = librosa.feature.mfcc(y=audio_denoised, sr=16000, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    rmse = librosa.feature.rms(y=audio_denoised, frame_length=n_fft, hop_length=hop_length)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio_denoised, sr=16000, n_fft=n_fft, hop_length=hop_length)

    # Calculate mean values
    mean_mfcc = np.mean(mfccs, axis=1)
    mean_rmse = np.mean(rmse)
    mean_centroid = np.mean(spectral_centroid)

    return np.concatenate([mean_mfcc, [mean_rmse], [mean_centroid]]).reshape(1, -1)

# Callback function for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print(f"Connected with result code {rc}")
    # Subscribe to the topic "topic/2/value"
    client.subscribe("topic/2/value")

# Callback function for when the client receives a message from the server.
def on_message(client, userdata, msg):
    print(f"Received message: {msg.payload}")
    # Convert the received array string to integer array
    received_array = np.array(msg.payload.decode("utf-8").split(','), dtype=int)
    # Preprocess the received audio data
    sound_features = preprocess_audio(received_array)
    # Make prediction using the SVM model
    prediction = svm_model.predict(sound_features)
    # Display the classification result
    print('Sound Classification:', "Scream" if prediction[0] == 1 else "Conversation")
    # Publish the processed data to the topic "topic/1/value"
    client.publish("topic/1/value", str(prediction[0]))

# Set MQTT broker address
broker_address = "192.168.43.243"  # Ganti dengan alamat IP broker MQTT EMQX

# Create a MQTT client instance
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1)
# Set the callback functions
client.on_connect = on_connect
client.on_message = on_message

# Connect to the MQTT broker
client.connect(broker_address, 1883, 60)

# Start the MQTT loop
client.loop_forever()


<ipython-input-1-df4656a0157c>:76: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1)


Connected with result code 0
Received message: b'504,312,401,432,482,445,478,397,393,442,418,446,458,448,448,447,446,447,447,446,2'


d:\Data\Anaconda\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 6 is too high: all coefficients will experience boundary effects.
  warnings.warn(
d:\Data\Anaconda\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=22
  warnings.warn(


Sound Classification: Scream
Received message: b'481,457,385,554,391,428,444,470,457,498,376,391,441,438,446,448,446,447,446,446,2'
Sound Classification: Scream


d:\Data\Anaconda\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 6 is too high: all coefficients will experience boundary effects.
  warnings.warn(
d:\Data\Anaconda\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=22
  warnings.warn(


...

Received message: b'516,440,447,444,446,447,446,447,445,447,446,446,447,446,446,446,447,446,446,446,2'
Sound Classification: Scream


d:\Data\Anaconda\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 6 is too high: all coefficients will experience boundary effects.
  warnings.warn(
d:\Data\Anaconda\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=22
  warnings.warn(


Received message: b'510,416,446,507,357,427,432,486,465,471,468,459,435,427,436,446,448,446,447,446,2'
Sound Classification: Scream


d:\Data\Anaconda\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 6 is too high: all coefficients will experience boundary effects.
  warnings.warn(
d:\Data\Anaconda\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=22
  warnings.warn(


KeyboardInterrupt: 

In [2]:
import os
import numpy as np
import librosa
import pywt
import joblib
import paho.mqtt.client as mqtt
from sklearn.preprocessing import StandardScaler

# Constants for feature extraction
N_MFCC = 13
N_FFT = 2048
HOP_LENGTH = 512

# Load the trained SVM model
model_filename = 'suara_model_svm.pkl'
svm_model = joblib.load(model_filename)

# Function for denoising using wavelet
def denoise_wavelet(audio, threshold=1e-6, preserve_threshold=0.05):
    coeffs = pywt.wavedec(audio, 'db1', level=6)
    preserved_coeffs = [coeffs[0]]
    
    for i in range(1, len(coeffs)):
        thresholded_coeff = pywt.threshold(coeffs[i], threshold, mode='soft')
        if np.sum(np.abs(thresholded_coeff)) > preserve_threshold:
            preserved_coeffs.append(thresholded_coeff)
        else:
            preserved_coeffs.append(np.zeros_like(thresholded_coeff))

    audio_denoised = pywt.waverec(preserved_coeffs, 'db1')
    return audio_denoised

# Function to preprocess audio and extract features
def preprocess_audio(audio_data, sr=16000, n_mfcc=N_MFCC, n_fft=N_FFT, hop_length=HOP_LENGTH):
    # Normalize sensor data to match the range of audio data
    audio = 2 * (audio_data - np.min(audio_data)) / (np.max(audio_data) - np.min(audio_data)) - 1

    # Denoising using wavelet
    audio_denoised = denoise_wavelet(audio, preserve_threshold=0.05)

    # Extracting features
    mfccs = librosa.feature.mfcc(y=audio_denoised, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    rmse = librosa.feature.rms(y=audio_denoised, frame_length=n_fft, hop_length=hop_length)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio_denoised, sr=sr, n_fft=n_fft, hop_length=hop_length)

    # Calculate mean values
    mean_mfcc = np.mean(mfccs, axis=1)
    mean_rmse = np.mean(rmse)
    mean_centroid = np.mean(spectral_centroid)

    return np.concatenate([mean_mfcc, [mean_rmse], [mean_centroid]]).reshape(1, -1)

# Callback function for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print(f"Connected with result code {rc}")
    # Subscribe to the topic "topic/2/value"
    client.subscribe("topic/2/value")

# Callback function for when the client receives a message from the server.
def on_message(client, userdata, msg):
    print(f"Received message: {msg.payload}")
    # Convert the received array string to integer array
    received_array = np.array(msg.payload.decode("utf-8").split(','), dtype=int)
    # Preprocess the received audio data
    sound_features = preprocess_audio(received_array)
    # Make prediction using the SVM model
    prediction = svm_model.predict(sound_features)
    # Display the classification result
    print('Sound Classification:', "Scream" if prediction[0] == 1 else "Conversation")
    # Publish the processed data to the topic "topic/1/value"
    client.publish("topic/1/value", str(prediction[0]))

# Set MQTT broker address
broker_address = "192.168.43.243"  # Ganti dengan alamat IP broker MQTT EMQX

# Create a MQTT client instance
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1)
# Set the callback functions
client.on_connect = on_connect
client.on_message = on_message

# Connect to the MQTT broker
client.connect(broker_address, 1883, 60)

# Start the MQTT loop
client.loop_forever()


<ipython-input-2-211ec4b6e31a>:77: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1)


Connected with result code 0
Received message: b'495,360,447,512,406,410,362,428,447,449,446,446,447,446,446,446,446,447,446,446,2'
Sound Classification: Conversation


d:\Data\Anaconda\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 6 is too high: all coefficients will experience boundary effects.
  warnings.warn(
d:\Data\Anaconda\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=22
  warnings.warn(


Received message: b'616,459,541,357,447,440,449,446,447,447,447,447,446,446,446,446,436,446,446,446,2'
Sound Classification: Conversation


d:\Data\Anaconda\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 6 is too high: all coefficients will experience boundary effects.
  warnings.warn(
d:\Data\Anaconda\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=22
  warnings.warn(


...

Received message: b'499,511,469,465,436,485,415,365,349,468,422,434,428,413,444,438,449,445,447,446,2'
Sound Classification: Conversation


d:\Data\Anaconda\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 6 is too high: all coefficients will experience boundary effects.
  warnings.warn(
d:\Data\Anaconda\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=22
  warnings.warn(


Received message: b'500,348,703,411,492,375,486,429,384,368,447,440,445,442,447,445,445,445,445,442,2'
Sound Classification: Conversation


d:\Data\Anaconda\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 6 is too high: all coefficients will experience boundary effects.
  warnings.warn(
d:\Data\Anaconda\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=22
  warnings.warn(


KeyboardInterrupt: 